In [15]:
from ultralytics import YOLO
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
import IPython
from utils import visualize_annotations, classname_to_color_mapper, draw_prediction_on_face
from tqdm.notebook import tqdm

def imshow(img):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret,width=1000,height=1000)
    IPython.display.display(i)

def delete_all_files_in_folder(folder_path, recursivity = True):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                if recursivity:
                    delete_all_files_in_folder(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

model = YOLO('runs/segment/train_export_v6_large/weights/best.pt')  # load a pretrained model (recommended for training)

padding_list = [50,100,150,200,250,300,350,400]
#img_name_list = [   "cubemap_P74_60e7f7bbe4021bd5e092ba94d6d502b7",
#                    "cubemap_P74_6306ff46d0032d2541caf3af9d9d08b9",
#                    "cubemap_P74_a38108830dd4372900c7a6f3281cc63b",
#                    "cubemap_P74_df1ea7387710ea3982cef1c05cfbcfac",]

img_name_list = [   "cubemap_P74_a5859d67c047bc97df9f718622e65a0e",
                    "cubemap_P74_01f8320ffe4c8f2dc11a8e161e287a62",
                    "cubemap_P74_63487d1f27267d9dc203362807e73e18",
                    "cubemap_P74_bbbe5b7e4493442f2b069645c4882a93",]

img_name_list = [   "cubemap_P74_01f8320ffe4c8f2dc11a8e161e287a62",]


if(args_are_ok(img_name_list, padding_list)):
    create_padded_predictions(img_name_list, padding_list)



  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
def create_padded_predictions(img_name_list, padding_value_list):

    for img_name in (p_bar := tqdm(img_name_list)):
        

        
        #p_bar.set_description(f"loading image {img_name}")

        img_name_posx = img_name+'_posx'
        img_name_negx = img_name+'_negx'
        img_name_posy = img_name+'_posy'
        img_name_negy = img_name+'_negy'
        img_name_posz = img_name+'_posz'
        img_name_negz = img_name+'_negz'
        face_path_list = [img_name_posx,img_name_negx,img_name_posy,img_name_negy,img_name_posz,img_name_negz]
        #results = model('datasets/Corrosao/images_full/2174.png', save = True)  # predict on an image

        # Load your RGB image
        image = Image.open('SISTEMAS_UFF/Images/cubemap_P74_fb48bf8c7a98b06384f1f93fcf027adb_posx.png')
        width,height = image.size

        image = image.convert("RGBA")


        
        # Create a new image with an alpha channel
        image_with_alpha = Image.new("RGBA", image.size)


        complete_image = np.zeros((height*3,width*4,4))

        # Create a drawing context for the image with an alpha channel
        draw = ImageDraw.Draw(image_with_alpha)

        # Define the points that form the polygon (replace with your polygon points)
        polygon_points = [(100, 100), (200, 100), (200, 200), (100, 200)]

        # Specify the fill color with transparency (e.g., (R, G, B, A))
        fill_color = (255, 0, 0, 128)  # Red polygon with 50% opacity


        draw_bounding_box = False

        # Draw the polygon on the image with transparency

        for face_name in face_path_list:
            

            image_original = Image.open(f"SISTEMAS_UFF/Images/{face_name}.png")
            #image_original = Image.open(f"datasets/darwin_export_holes/images/test/{face_name}.png")

            if face_name.endswith("PosX"):
                pass
            elif face_name.endswith("NegX"):
                pass
            elif face_name.lower().endswith("negy"):
                image_original = image_original.rotate(270)
                pass
            elif face_name.lower().endswith("posy"):
                image_original = image_original.rotate(90)
                pass
            elif face_name.endswith("PosZ"):
                pass
            elif face_name.endswith("NegZ"):
                pass
            
            results = model(image_original, save = False, verbose=False)  # predict on an image
            
            image = image_original.convert("RGBA")

            # Create a new image with an alpha channel
            image_with_alpha = Image.new("RGBA", image.size)
            if draw_bounding_box:
                image_with_alpha_bb = Image.new("RGBA", image.size)

            # Create a drawing context for the image with an alpha channel
            draw = ImageDraw.Draw(image_with_alpha)
            
            if draw_bounding_box:
                draw_bb = ImageDraw.Draw(image_with_alpha_bb)
            for result in results:
                
                if len(result) > 0:

                    masks = result.masks.xy  # Masks object for segmentation masks outputs
                    boxes = result.boxes.xyxy  # Box object for bounding box outputs
                    classes = result.boxes.cls  # Class object for segmentation masks outputs
                    for i in range(len(masks)):
                        mask = masks[i]
                        box = boxes[i]
                        classe_index = int(classes[i].item())
                        classe_name = result.names[classe_index]

                        fill_color = classname_to_color_mapper(classe_name)
                        outline_color = classname_to_color_mapper(classe_name,255)

                        polygon_point_list = []
                        bb_point_list = []
                        for point in mask:
                            x,y = point
                            polygon_point_list.append((x,y))

                        x_min = box[0].item()
                        x_max = box[2].item()
                        y_min = box[1].item()
                        y_max = box[2].item()

                        bb_point_list.append((x_min,y_min))
                        bb_point_list.append((x_min,y_max))
                        bb_point_list.append((x_max,y_max))
                        bb_point_list.append((x_max,y_min))
                            
                        if len(polygon_point_list) > 0:
                            draw.polygon(polygon_point_list, fill_color, outline_color)
                        else:
                            print(f"polygon_point_list:{polygon_point_list}")
                            print(f"bb_point_list:{bb_point_list}",end="\n\n")

                        if len(bb_point_list) > 0 and draw_bounding_box:
                            draw_bb.polygon(bb_point_list, (0,0,0,0),outline_color, width=10)
                        else:
                            if draw_bounding_box:
                                print(f"polygon_point_list:{polygon_point_list}")
                                print(f"bb_point_list:{bb_point_list}",end="\n\n")
                            
                annotated_face = Image.alpha_composite(image, image_with_alpha)
                if draw_bounding_box:
                    annotated_face = Image.alpha_composite(annotated_face, image_with_alpha_bb)
                img = np.array(annotated_face) 
                
            height_pos = 0
            width_pos = 0
            if face_name.endswith("posx"):
                height_pos = 1
                width_pos = 2
                pass
            elif face_name.endswith("negx"):
                height_pos = 1
                width_pos = 0
                pass
            elif face_name.endswith("negy"):
                height_pos = 0
                width_pos = 1
                pass
            elif face_name.endswith("posy"):
                height_pos = 2
                width_pos = 1
                pass
            elif face_name.endswith("posz"):
                height_pos = 1
                width_pos = 1
                pass
            elif face_name.endswith("negz"):
                height_pos = 1
                width_pos = 3
                pass
            complete_image[height*height_pos:height*(height_pos+1),width*width_pos:width*(width_pos+1),:] = img
        complete_image[..., :3] = complete_image[..., 2::-1]

        #visualize_annotations(img_name,f"SISTEMAS_UFF/corrosion-v3/{img_name[8:]}.json")
        #imshow(complete_image)
        directory = os.path.join("face_border_problem_ideia",face_name[:-5])
        if not os.path.exists(directory):
            os.makedirs(directory)
        else:
            delete_all_files_in_folder(directory, False)

        im = Image.fromarray(complete_image[..., [2,1,0]].astype(np.uint8))
        im.save(os.path.join(directory,"original.png"))


        #visualize_annotations(img_name,f"SISTEMAS_UFF/export-v6/{img_name[8:]}.json")

        img_name_posx = img_name+'_posx'
        img_name_negx = img_name+'_negx'
        img_name_posy = img_name+'_posy'
        img_name_negy = img_name+'_negy'
        img_name_posz = img_name+'_posz'
        img_name_negz = img_name+'_negz'
        face_path_list = [img_name_posx,img_name_negx,img_name_posy,img_name_negy,img_name_posz,img_name_negz]
        #results = model('datasets/Corrosao/images_full/2174.png', save = True)  # predict on an image

        image = Image.open('SISTEMAS_UFF/Images/cubemap_P74_fb48bf8c7a98b06384f1f93fcf027adb_posx.png')
        width,height = image.size

        complete_image = np.zeros((height*3,width*4,3))


        for face_name in face_path_list:
            image_original = Image.open(f"SISTEMAS_UFF/Images/{face_name}.png")
            #image_original = Image.open(f"datasets/darwin_export_holes/images/test/{face_n
            
            if face_name.endswith("PosX"):
                pass
            elif face_name.endswith("NegX"):
                pass
            elif face_name.lower().endswith("negy"):
                image_original = image_original.rotate(270)
                pass
            elif face_name.lower().endswith("posy"):
                image_original = image_original.rotate(90)
                pass
            elif face_name.endswith("PosZ"):
                pass
            elif face_name.endswith("NegZ"):
                pass
            
            img = np.array(image_original) 
            height_pos = 0
            width_pos = 0
            if face_name.endswith("posx"):
                height_pos = 1
                width_pos = 2
                pass
            elif face_name.endswith("negx"):
                height_pos = 1
                width_pos = 0
                pass
            elif face_name.endswith("negy"):
                height_pos = 0
                width_pos = 1
                pass
            elif face_name.endswith("posy"):
                height_pos = 2
                width_pos = 1
                pass
            elif face_name.endswith("posz"):
                height_pos = 1
                width_pos = 1
                pass
            elif face_name.endswith("negz"):
                height_pos = 1
                width_pos = 3
                pass
            complete_image[height*height_pos:height*(height_pos+1),width*width_pos:width*(width_pos+1),:] = img
        complete_image[..., :3] = complete_image[..., 2::-1]

        #visualize_annotations(img_name,f"SISTEMAS_UFF/corrosion-v3/{img_name[8:]}.json")
        #imshow(complete_image)





        result = model(complete_image, verbose=False)

        prediction_on_full_image = draw_prediction_on_face(complete_image[..., [2,1,0]],result[0])

        #imshow(prediction_on_full_image)

        im = Image.fromarray(complete_image[..., [2,1,0]].astype(np.uint8))
        im.save(os.path.join(directory,"original_image.png"))


        #-----------------------------------------


        im = Image.fromarray(prediction_on_full_image[..., [2,1,0]].astype(np.uint8))
        im.save(os.path.join(directory,"predict_on_full_image.png"))



        cropped_image  = complete_image[:height*2,:width*2]

        result = model(cropped_image, verbose=False)

        prediction_on_cropped_image = draw_prediction_on_face(cropped_image[..., [2,1,0]],result[0])

        #imshow(prediction_on_cropped_image)

        im = Image.fromarray(prediction_on_cropped_image[..., [2,1,0]].astype(np.uint8))
        im.save(os.path.join(directory,"predict_on_multiple_faces.png"))
        cropped_image  = np.copy(complete_image)[:height*3,:width*3]

        #setting reflections
        cropped_image[height*0:height*1,width*1:width*2] = complete_image[height*2:height*1:-1,width*1:width*2]
        cropped_image[height*2:height*3,width*1:width*2] = complete_image[height*2:height*1:-1,width*1:width*2]

        cropped_image[height*1:height*2,width*0:width*1] = complete_image[height*1:height*2,width*2:width*1:-1]
        cropped_image[height*1:height*2,width*2:width*3] = complete_image[height*1:height*2,width*2:width*1:-1]

        ground_truth = visualize_annotations(img_name,f"SISTEMAS_UFF/export-v6/{img_name[8:]}.json", True)
        
        im = Image.fromarray(ground_truth[...,[2,1,0]].astype(np.uint8))
        im.save(os.path.join(directory,"ground_truth.png"))

        
        for i in range(len(padding_value_list)):

            horizontal_padding = padding_value_list[i]
            vertical_padding = padding_value_list[i]
            
            directory_padding = os.path.join(directory,f"padding_{horizontal_padding}")


            if not os.path.exists(directory_padding):
                os.makedirs(directory_padding)
            else:
                delete_all_files_in_folder(directory_padding, False)

            reflected_image = cropped_image[height*1-vertical_padding:height*2+vertical_padding,width*1-horizontal_padding:width*2+horizontal_padding]
            #imshow(reflected_image)


            im = Image.fromarray(reflected_image[..., [2,1,0]].astype(np.uint8))
            im.save(os.path.join(directory,"reflected_image.png"))

            result = model(reflected_image, verbose=False)

            prediction_on_reflected_image = draw_prediction_on_face(reflected_image[..., [2,1,0]],result[0])

            #imshow(prediction_on_reflected_image)

            im = Image.fromarray(prediction_on_reflected_image[..., [2,1,0]].astype(np.uint8))
            im.save(os.path.join(directory_padding,f"prediction_on_reflected_image_{horizontal_padding}.png"))

            #imshow(prediction_on_reflected_image[vertical_padding:height,horizontal_padding:width])


            im = Image.fromarray(prediction_on_reflected_image[vertical_padding:height,horizontal_padding:width, [2,1,0]].astype(np.uint8))
            im.save(os.path.join(directory_padding,f"prediction_on_reflected_image_cropped_{horizontal_padding}.png"))
            cropped_image  = np.zeros(((height*3)+(2*vertical_padding),width*3+(2*horizontal_padding),3))
            predicted_reflected_image_full  = np.zeros((height*3,width*4,3))

            for y_spot in range(3):
                for x_spot in range(4):
                    
                    #setting reflections
                    face_to_reflect = complete_image[height*(y_spot):height*(y_spot+1),width*(x_spot):width*(x_spot+1)]
                    #print(face_to_reflect.shape)
                    cropped_image[height*0:height*1,width*1:width*2] = face_to_reflect[::-1]
                    cropped_image[height*2:height*3,width*1:width*2] = face_to_reflect[::-1]

                    cropped_image[height*1:height*2,width*0:width*1] = face_to_reflect[:,::-1]
                    cropped_image[height*1:height*2,width*2:width*3] = face_to_reflect[:,::-1]

                    cropped_image[height*1:height*2,width*1:width*2] = face_to_reflect

                    reflected_image = cropped_image[height*1-vertical_padding:height*2+vertical_padding,width*1-horizontal_padding:width*2+horizontal_padding]
                    #imshow(reflected_image)


                    #im = Image.fromarray(reflected_image[..., [2,1,0]].astype(np.uint8))
                    #im.save(os.path.join(directory,"reflected_image.png"))

                    result = model(reflected_image, verbose=False)

                    prediction_on_reflected_image = draw_prediction_on_face(reflected_image[..., [2,1,0]],result[0])

                    prediction_on_reflected_image_cropped = prediction_on_reflected_image[vertical_padding:height+vertical_padding,horizontal_padding:width+horizontal_padding,0:3]
                    predicted_reflected_image_full[height*(y_spot):height*(y_spot+1),width*(x_spot):width*(x_spot+1)] = prediction_on_reflected_image_cropped

                    #imshow(prediction_on_reflected_image)
                    #im = Image.fromarray(prediction_on_reflected_image[..., [2,1,0]].astype(np.uint8))
                    #im.save(os.path.join(directory,"prediction_on_reflected_image.png"))
                    #imshow(prediction_on_reflected_image[vertical_padding:height,horizontal_padding:width])
                    #im = Image.fromarray(prediction_on_reflected_image[vertical_padding:height,horizontal_padding:width, [2,1,0]].astype(np.uint8))
                    #im.save(os.path.join(directory,"predict_on_multiple_faces_cropped.png"))
            #imshow(predicted_reflected_image_full)

            im = Image.fromarray(predicted_reflected_image_full[...,[2,1,0]].astype(np.uint8))
            im.save(os.path.join(directory_padding,f"predict_on_all_reflected_faces_complete_{horizontal_padding}.png"))
            #im.save(os.path.join(directory,f"predict_on_all_reflected_faces_complete_{horizontal_padding}.png"))
            
            canvas = np.zeros((height*3,width*3,3))
            prediction_on_multiple_faces = np.zeros((height*3,width*4,4))

            topo     = np.s_[height*0:height*1,width*1:width*2]
            esquerda = np.s_[height*1:height*2,width*0:width*1]
            centro   = np.s_[height*1:height*2,width*1:width*2]
            direita  = np.s_[height*1:height*2,width*2:width*3]
            contrario  = np.s_[height*1:height*2,width*3:width*4]
            base     = np.s_[height*2:height*3,width*1:width*2]

            lista_faces = [esquerda,centro,direita,contrario]

            face_1 = complete_image[esquerda]
            face_2 = complete_image[centro]
            face_3 = complete_image[direita]
            face_4 = complete_image[contrario]

            faces = [face_1,face_2,face_3,face_4]


            face_topo = complete_image[topo]
            face_base = complete_image[base]



            for face_number in range(len(faces)):
                
                canvas[topo]     = np.rot90(face_topo, k=face_number, axes=(1,0))
                canvas[esquerda] = np.rot90(faces[face_number], k=0, axes=(1,0))
                canvas[centro]   = np.rot90(faces[(face_number+1)%4], k=0, axes=(1,0))
                canvas[direita]  = np.rot90(faces[(face_number+2)%4], k=0, axes=(1,0))
                canvas[base]     = np.rot90(face_base, k=-face_number, axes=(1,0))

                #imshow(canvas)
                #result = model(canvas[:,:])
                result = model(canvas[height*1-vertical_padding:height*2+vertical_padding,width*1-horizontal_padding:width*2+horizontal_padding], verbose=False)
                
                prediction_on_reflected_image = draw_prediction_on_face(canvas[..., [2,1,0]][height*1-vertical_padding:height*2+vertical_padding,width*1-horizontal_padding:width*2+horizontal_padding],result[0])
                prediction_on_multiple_faces[lista_faces[(face_number+1)%4]] = prediction_on_reflected_image[vertical_padding:height+vertical_padding,horizontal_padding:width+horizontal_padding]
                #imshow(prediction_on_multiple_faces)
                

            canvas[topo]     = np.rot90(faces[3], k=2, axes=(1,0))
            canvas[esquerda] = np.rot90(faces[0], k=1, axes=(1,0))
            canvas[centro]   = np.rot90(face_topo, k=0, axes=(1,0))
            canvas[direita]  = np.rot90(faces[2], k=-1, axes=(1,0))
            canvas[base]     = np.rot90(faces[1], k=-0, axes=(1,0))

            result = model(canvas[height*1-vertical_padding:height*2+vertical_padding,width*1-horizontal_padding:width*2+horizontal_padding], verbose=False)
                
            prediction_on_reflected_image = draw_prediction_on_face(canvas[..., [2,1,0]][height*1-vertical_padding:height*2+vertical_padding,width*1-horizontal_padding:width*2+horizontal_padding],result[0])
            prediction_on_multiple_faces[topo] = prediction_on_reflected_image[vertical_padding:height+vertical_padding,horizontal_padding:width+horizontal_padding]


            #imshow(prediction_on_multiple_faces)

            canvas[topo]     = np.rot90(faces[1], k=0, axes=(1,0))
            canvas[esquerda] = np.rot90(faces[0], k=-1, axes=(1,0))
            canvas[centro]   = np.rot90(face_base, k=0, axes=(1,0))
            canvas[direita]  = np.rot90(faces[2], k=1, axes=(1,0))
            canvas[base]     = np.rot90(faces[3], k=2, axes=(1,0))

            result = model(canvas[height*1-vertical_padding:height*2+vertical_padding,width*1-horizontal_padding:width*2+horizontal_padding], verbose=False)
                
            prediction_on_reflected_image = draw_prediction_on_face(canvas[..., [2,1,0]][height*1-vertical_padding:height*2+vertical_padding,width*1-horizontal_padding:width*2+horizontal_padding],result[0])
            prediction_on_multiple_faces[base] = prediction_on_reflected_image[vertical_padding:height+vertical_padding,horizontal_padding:width+horizontal_padding]
            #imshow(prediction_on_multiple_faces)

            im = Image.fromarray(prediction_on_multiple_faces[...,[2,1,0]].astype(np.uint8))
            #im.save(os.path.join(directory_padding,f"prediction_with_4_faces_{horizontal_padding}.png"))
            im.save(os.path.join(directory,f"prediction_with_4_faces_{horizontal_padding}.png"))

            #imshow(prediction_on_multiple_faces)
            #imshow(predicted_reflected_image_full)
            #imshow(ground_truth)
            #imshow(complete_image)

In [2]:
def args_are_ok(img_name_list, padding_value_list):
    if type(img_name_list) is not list or type(img_name_list) is not list:
        print(f"Arguments should be lists but are {type(img_name_list)} and {type(img_name_list)}")
        return False
        
    for i in range(len(img_name_list)):
        img_name = img_name_list[i]

        if type(img_name) is not str:
            print(f"All values of img_name_list should be strings not {type(img_name)}")
            return False
        padding = padding_value_list[i]

        if type(padding) is not int:
            print(f"All values of padding_value_list should be integers, but is {type(padding)}")
            return False

        if padding < 0:
            print(f"All values of padding_value_list should be 0 or bigger, but is {padding}")
            return False
    return True
        